In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
This notebook demonstrates how to use the Crop Data Layers (CDL) API 
to download pixel-level crop type data for various bounding boxes and years.
"""

import sys
import os

# Set up imports
project_root = os.path.abspath("..") 
sys.path.append(project_root)  

# GPU Config
from config.gpu.gpu_utils import check_GPU_config

gpu_device = check_GPU_config()
print(f"GPU config: {gpu_device}")

In [ ]:
from preprocessing.datasources import (
    map_cdl_codes_to_rgb_and_text,
    CDL_clip_retrieve,
    replace_matrix_elements,
    read_bytes_plot_clipped_CDL_image
    )

# Call the function to obtain the code to RGB mapping and code to text mapping
code_to_rgb_mapping_dict, code_to_text_mapping_dict, code_mapping_df = map_cdl_codes_to_rgb_and_text()
print(code_to_rgb_mapping_dict)
print(code_to_text_mapping_dict)

In [ ]:
code_mapping_df.head()

In [ ]:
import numpy as np

# Example usage
matrix = np.array([
    [1, 2, 3],
    [3, 1, 2],
    [2, 3, 1]
])

color_dict = {
    1: (255, 0, 0),  # Red
    2: (0, 255, 0),  # Green
    3: (0, 0, 255),  # Blue
}

new_matrix = replace_matrix_elements(matrix, color_dict, code_to_text_mapping_dict)
new_matrix

### 🌱 Mapping Crop Codes to RGB and Text

Each value in the matrix is a **crop code**.  
The new matrix stores the **RGB** and **text** combination for each code.  

#### 📌 Example:

```python
# Matrix with base codes
np.array([
    [1, 2, 3],
    [3, 1, 2],
    [2, 3, 1]
])

# RGB Representation of Each Crop Code
array([
    [[255, 0, 0], [0, 255, 0], [0, 0, 255]],  # Red, Green, Blue
    [[0, 0, 255], [255, 0, 0], [0, 255, 0]],  # Blue, Red, Green
    [[0, 255, 0], [0, 0, 255], [255, 0, 0]]   # Green, Blue, Red
])

# Text Representation of Each Crop Code
array([
    ['Corn', 'Cotton', 'Rice'],
    ['Rice', 'Corn', 'Cotton'],
    ['Cotton', 'Rice', 'Corn']
], dtype=object)



# Define AOIs and visualize/test retreival via interactive plots

In [ ]:
# A list of bounding boxes and years that we manually identified around the lower Mississippi delta as candidate 
# areas for training and validation demo. NASSGEO API requires the bounds in "EPSG:5070" in the form of left, bottom, right, top. A conversion 

bbox_list = [
    '426362, 1405686, 520508, 1432630', 
    '390747, 1195097, 437820, 1284288', 
    '465073, 1479393, 583994, 1504168', 
    '549309, 1536066, 592356, 1571061', 
    '491706, 1510362, 571916, 1531111', 
    '434104, 1306585, 498520, 1342509', 
    '414748, 1149262, 439833, 1193858'
    ]

# A smaller area and year in which data will be sampled at full resolution for a final test (the above areas will be subsampled spatially)
bbox_unique_scene = '484932, 1401912, 489035, 1405125'
year_unique_scene = 2019

In [ ]:
# Load CDL data
external_drive_path = "/media/{external_path_to}/cropdata/US" # fallback

try:
    tif_bytes_out = CDL_clip_retrieve(
        bbox=bbox_list[0], # if API is down, bbox can be also ignored with None
        year=2019,  
        local_path=external_drive_path
    )
    print("✅ Data ready for processing.")
except Exception as e:
    print(f"⚠️ Final error: {e}")

In [ ]:
# plot sample window
read_bytes_plot_clipped_CDL_image(tif_bytes_out, window_size=800, sample_region=True)

In [ ]:
'''
We need to ignore double cropped regions during training due to complexity of labeling...
but will be interesting to see what the algorithm predicts on this dense real-time test for those areas.
'''
tif_bytes_out_unique_scene = CDL_clip_retrieve(bbox_unique_scene, year_unique_scene)
read_bytes_plot_clipped_CDL_image(tif_bytes_out_unique_scene, window_size=None)

# Create and visually check Subsampling functions

In [ ]:
import rasterio
from io import BytesIO

from preprocessing.datasources import (
    sample_raster_data,
    create_matrix_from_sampled_data,
    plot_CDL_clipped_img
    )

import numpy as np
from plotly.subplots import make_subplots


In [ ]:
'''
Observe sampling CDL operated as expected (downsampled raster values correctly). "Background" pixels here are areas where sampling was skipped.
'''

# Load the original raster and get its shape
with rasterio.io.MemoryFile(BytesIO(tif_bytes_out)) as memfile:
    with memfile.open() as dataset:
        original_shape = dataset.read(1).shape
        
# Sample the raster, and create a new raster from the sampled data
sampled_data, longitudes, latitudes = sample_raster_data(tif_bytes_out, interval=3)
downsampled_raster_data = create_matrix_from_sampled_data(sampled_data, original_shape)

# Crear una figura con una sola subgráfica
fig = make_subplots(rows=1, cols=1, subplot_titles=["Downsampled CDL Image"])

# Plot the new raster data
rgb_image, text_matrix = replace_matrix_elements(downsampled_raster_data, code_to_rgb_mapping_dict, code_to_text_mapping_dict)
plot_CDL_clipped_img(downsampled_raster_data[:800, :800], rgb_image[:800, :800], text_matrix[:800, :800], fig, row=1, col=1)

# Mostrar la figura después de agregar las trazas
fig.update_layout(title="Downsampled CDL Data", width=600, height=600)
fig.show()

# Calculate pixels corresponding to 'Corn'
pixel_count_corn = np.count_nonzero(text_matrix[:800, :800] == 'Corn')
print(f"The 'Corn' corresponds to {pixel_count_corn} pixels")


In [ ]:
import matplotlib.pyplot as plt 

# Check that downsampled data covers the same lat/lon extents as original data, but is truly taking a subset of the locations
_, orig_lons, orig_lats = sample_raster_data(tif_bytes_out, interval=1)
# Plot the raster data and sampled points
plt.scatter(orig_lons, orig_lats, color='blue', alpha=1, s=1)
plt.scatter(longitudes, latitudes, color='red', s=1, alpha=1)
plt.gca().set_xlim(left=orig_lons.min(), right=orig_lons.min()+0.002)
plt.gca().set_ylim(bottom=orig_lats.min(), top=orig_lats.min()+0.08)
plt.show()

plt.figure()
plt.scatter(orig_lons, orig_lats, color='blue', s=.001)
plt.scatter(longitudes, latitudes, color='red', s=.001)
plt.show()

# Get the data

In [ ]:
years = [2019, 2020, 2021]
tif_bytes_list = [[CDL_clip_retrieve(x, y) for x in bbox_list] for y in years]

In [ ]:
import pandas as pd
from datasources import (
    get_crop_stats_from_AOI,
    display_table
)

stats_list = [
    [
        (*get_crop_stats_from_AOI(t, code_to_rgb_mapping_dict, code_to_text_mapping_dict), s[-1]) 
        for t in s[0]
    ]
    for s in zip(tif_bytes_list, years)  
]


# Concatenating the DataFrames
df_stats_list = pd.concat([display_table(x) for x in stats_list], ignore_index=True)
display(df_stats_list)

In [ ]:
display(df_stats_list.Year.unique()) 

In [ ]:
'''
Visualize a subsampled scene to make sure it looks reasonable
'''
# Sample the raster, and create a new raster from the sampled data
sampled_data, longitudes, latitudes = sample_raster_data(tif_bytes_list[0][0], interval=10)

# Plot the new raster data
rgb_image, text_matrix = replace_matrix_elements(sampled_data, code_to_rgb_mapping_dict, code_to_text_mapping_dict)

fig = make_subplots(rows=1, cols=1, subplot_titles=["Subsampled CDL Image"])
plot_CDL_clipped_img(sampled_data[:, :], rgb_image[:, :], text_matrix[:, :], fig, row=1, col=1)

fig.show()

# calculate pixels corresponding to 'Corn'
pixel_count_corn = np.count_nonzero(text_matrix[:, :] == 'Corn')
print(f"The 'Corn' corresponds to {pixel_count_corn} pixels")

# Sample Data & Write the table!

In [ ]:
from spark_session import spark
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datasources import sample_data_into_dataframe_write_parquet

# Save a "dense" test dataset
sample_data_into_dataframe_write_parquet(tif_bytes_out_unique_scene, bbox_unique_scene, year_unique_scene, 
                                         filename="CDL_unique_scene.parquet", interval=1)

# Quick check that the dataframe saved properly by loading & counting the number of rows
target_path = os.path.join(project_root, 'data/CDL_unique_scene.parquet/')

unique_scene_pt = spark.read.parquet(target_path)
unique_scene_pt.show(5)

#print(f"Total number of rows: {dense_parquet.count()}")
print(f"Total number of rows: {unique_scene_pt.count()}")

In [ ]:
'''
Write all training and validation data out. Downsample by factor of 15 in order to keep overall data size low 
since Sentinel-2 data for each pixel will increase data size significantly (>100x)
'''
for l1 in zip(tif_bytes_list, years):
    for l2 in zip(l1[0], bbox_list):
        sample_data_into_dataframe_write_parquet(l2[0], l2[1], l1[1], interval=15)

In [ ]:
'''
Load and display the train/validation data (using some aggregations) to check that it wrote out properly
'''
target_path = os.path.join(project_root, 'data/CDL_samples.parquet/')
sample_df = spark.read.parquet(target_path)
result = sample_df.groupBy("year", "CDL") \
    .agg(F.count("CDL").alias("count")) \
    .withColumn("total_count", F.sum("count").over(Window.partitionBy("year"))) \
    .withColumn("percentage", F.round((F.col("count") / F.col("total_count")) * 100, 2))
display(result)

In [ ]:
result.show(10, truncate=False)  # Show first 10 rows without truncating strings

In [ ]:
sample_df.select("bbox", "year").distinct().show()